    This code used to extract GLCM feature. Written by Muhammad Naufal, you can use it for educational purpose

Code below used to import all library needed to do feature extraction

In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy
import pandas as pd
import cv2

Code below used to define the datasets directory

In [2]:
img_dir= 'Crop ROI'

Code below used to create empty list for saving some variables such as images that contain an array, labels for image class, and the path of image

In [3]:
images=[]
labels=[]
img_paths=[]

Code below used to define the standard of image which is 64x64 pixel after resize

In [4]:
width = 64
height = 64
dim = (width, height)

Read all image and save the label and path

In [5]:
for subdir1 in os.listdir(img_dir):
    subdir2 = img_dir+'/'+subdir1
    for file in os.listdir(subdir2):
        file_path = subdir2+'/'+file
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, dim)
        images.append(img)
        labels.append(subdir1)
        img_paths.append(file_path)
        if cv2.waitKey(0) & 0xff == 27: 
            cv2.destroyAllWindows() 

Create empty list for collection of features of many combination on whole image

In [ ]:
dissimilarity=[]
homogeneity=[]
contrast=[]
ASM=[]
energy=[]
correlation=[]
entropy=[]

Code below used to extract the GLCM feature, the feature that being extracted are the combination of distance 1-5 & angles 0 - 135. It took some times, so you need to be patient to wait :)

In [ ]:
counter = 1
for img in images:
#     code below used to find cooccurence matrix for 20 combination of angles and distances
    matrix_cooc = greycomatrix(img, distances=[1, 2, 3, 4, 5], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256,
                        symmetric=True, normed=False)
    dissimilarity.append(greycoprops(matrix_cooc, 'dissimilarity'))
    homogeneity.append(greycoprops(matrix_cooc, 'homogeneity'))
    contrast.append(greycoprops(matrix_cooc, 'contrast'))
    ASM.append(greycoprops(matrix_cooc, 'ASM'))
    energy.append(greycoprops(matrix_cooc, 'energy'))
    correlation.append(greycoprops(matrix_cooc, 'correlation'))
    img_entropy = np.zeros((5, 4))
    for distance in range (0, 5):
        for angle in range(0, 4):
            img_entropy[distance][angle] = shannon_entropy(matrix_cooc[distance][angle])
    entropy = np.append(entropy, img_entropy)
    counter += 1
reshaped_entropy = np.reshape(entropy,(len(images), 5,4))

I define a function to export feature to csv, so after we run the code, we can save it and we don't have to re-run the code which took a long time to extract the feature. We can read the extracted feature after exported to csv

In [ ]:
def export_csv(distance, angle, dissimilarity, homogeneity, contrast, ASM, energy, correlation, entropy):
    datasets = {'dissimilarity': dissimilarity,
            'homogeneity': homogeneity,
            'contrast': contrast,
            'ASM': ASM,
            'energy': energy,
            'correlation': correlation,
            'entropy': entropy,
            'labels': labels,
            'img_paths': img_paths
           }
    file_name = 'glcm_'+str(distance)+'_'+str(angle)+'.csv'
    df = pd.DataFrame(datasets, columns = ['dissimilarity', 'homogeneity', 'contrast', 'ASM', 'energy', 'correlation', 'entropy', 'labels', 'img_paths'])
    df.to_csv(file_name, encoding='utf-8', index=False)

I called the csv export function for every combination

In [ ]:
for distance in range (0, 5):
    for angle in range(0, 4):
        single_dissimilarity=[]
        single_homogeneity=[]
        single_contrast=[]
        single_ASM=[]
        single_energy=[]
        single_correlation=[]
        single_entropy=[]
        for feature in range (0, len(dissimilarity)):
            single_dissimilarity.append(dissimilarity[feature][distance][angle])
            single_homogeneity.append(homogeneity[feature][distance][angle])
            single_contrast.append(contrast[feature][distance][angle])
            single_ASM.append(ASM[feature][distance][angle])
            single_energy.append(energy[feature][distance][angle])
            single_correlation.append(correlation[feature][distance][angle])
            single_entropy.append(reshaped_entropy[feature][distance][angle])
            
        single_dissimilarity= np.array(single_dissimilarity).flatten().tolist()
        single_homogeneity= np.array(single_homogeneity).flatten().tolist()
        single_contrast= np.array(single_contrast).flatten().tolist()
        single_ASM= np.array(single_ASM).flatten().tolist()
        single_energy= np.array(single_energy).flatten().tolist()
        single_correlation= np.array(single_correlation).flatten().tolist()
        single_entropy= np.array(single_entropy).flatten().tolist()
        export_csv(
                distance,
                angle,
                single_dissimilarity,
                single_homogeneity,
                single_contrast,
                single_ASM,
                single_energy,
                single_correlation,
                single_entropy
            )

# Zoning feature section

The code isn't kinda clean, it'll be run twice for 2 kinds of zoning from here

In [6]:
zoning_dissimilarity=[]
zoning_homogeneity=[]
zoning_entropy=[]

In [7]:
zoning_size = (4, 4)

In [8]:
def zoning(image, size):
    sub_images = []
    image_shape = image.shape
    image_range = int(image_shape[0]/zoning_size[0])
    for i in range(0, image_shape[0], image_range):
        for j in range (0, image_shape[1], image_range):
            sub_images.append(image[i:i+image_range, j:j+image_range])
    return sub_images

In [9]:
zoning_images = []

In [10]:
for image in images:
    zoning_images.append(zoning(image, zoning_size))

In [11]:
for zoned_image in zoning_images:
    for img in zoned_image:
#         matrix_cooc = greycomatrix(img, distances=[1, 2, 3, 4, 5], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256,
#                             symmetric=True, normed=False)
        matrix_cooc = greycomatrix(img, distances=[1], angles=[0], levels=256,
                            symmetric=True, normed=False)
        zoning_dissimilarity.append(greycoprops(matrix_cooc, 'dissimilarity'))
        zoning_homogeneity.append(greycoprops(matrix_cooc, 'homogeneity'))
        img_entropy = np.zeros((1, 1))
        for distance in range (0, 1):
            for angle in range(0, 1):
                img_entropy[distance][angle] = shannon_entropy(matrix_cooc[distance][angle])
        zoning_entropy = np.append(zoning_entropy, img_entropy)
reshaped_zoning_entropy = np.reshape(zoning_entropy,(len(images) * zoning_size[0]**2, 1,1))

In [12]:
index_start = 0;
index_jump = zoning_size[0]**2;
img_counter = 1;
datasets = []

for distance in range (0, 1):
    for angle in range(0, 1):
        while img_counter<=len(images):
            all_zoning_dissimilarity = []
            all_zoning_homogeneity = []
            all_zoning_entropy = []
            for sub_zoning in range (index_start, index_start+index_jump):
                all_zoning_dissimilarity.append(zoning_dissimilarity[sub_zoning][distance][angle])
                all_zoning_homogeneity.append(zoning_homogeneity[sub_zoning][distance][angle])
                all_zoning_entropy.append(reshaped_zoning_entropy[sub_zoning][distance][angle])


            all_zoning_dissimilarity = np.array(all_zoning_dissimilarity).flatten()
            all_zoning_homogeneity = np.array(all_zoning_homogeneity).flatten()
            all_zoning_entropy = np.array(all_zoning_entropy).flatten()
            
            datasets.append(np.concatenate((all_zoning_dissimilarity, all_zoning_homogeneity, all_zoning_entropy), axis=None))
            #keys based on range of column,  in case 2x2 zoning use 2*2*2, or 4x4 use 4*4*2 
            keys = range(2*zoning_size[0]**2)
            cols = []
            for sub_img in range (1, 1+zoning_size[0]**2):
                cols.append("8x8_Dissimilarity_"+str(sub_img))
            for sub_img in range (1, 1+zoning_size[0]**2):
                cols.append("8x8_Homogeneity_"+str(sub_img))
            for sub_img in range (1, 1+zoning_size[0]**2):
                cols.append("8x8_Entropy_"+str(sub_img))
                
            index_start += index_jump;
            img_counter +=1
        #reshape to -1, columnsize. in case 2x2 zoning use 2*2*2, or 4x4 use 4*4*2 
        datasets = np.array(datasets).reshape(-1, 3*zoning_size[0]**2)
        file_name = 'glcm_'+str(distance)+'_'+str(angle)+'_zoning size='+str(zoning_size[0])+'.csv'
        df = pd.DataFrame(datasets, columns = cols)
        df.to_csv(file_name, encoding='utf-8', index=False)